让我们回顾一下在 3.10节（“多层感知机的简洁实现”）一节中含单隐藏层的多层感知机的实现方法。我们首先构造`Sequential`实例，然后依次添加两个全连接层。其中第一层的输出大小为256，即隐藏层单元个数是256；第二层的输出大小为10，即输出层单元个数是10。我们在上一章的其他
节中也使用了`Sequential`类构造模型。这里我们介绍另外一种基于`tf.keras.Model`类的模型构造方法：它让模型构造更加灵活。

## 4.1.1 build model from block

`tf.keras.Model`类是`tf.keras`模块里提供的一个模型构造类，我们可以继承它来定义我们想要的模型。下面继承`tf.keras.Model`类构造本节开头提到的多层感知机。这里定义的`MLP`类重载了`tf.keras.Model`类的`__init__`函数和`call`函数。它们分别用于创建模型参数和**定义前向计算**。前向计算也即正向传播。

In [1]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

2.3.0


In [2]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()  # 简单的说super().__init__()，就是继承父类的init方法，同样可以使用super()去继承其他方法。
        self.flatten = tf.keras.layers.Flatten() # Flatten层将除第一维（batch_size）以外的维度展平
        self.dense1 = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)
        
    def call(self,inputs):
        x = self.flatten(inputs)
        x = self.dense1(x)
        output = self.dense2(x)
        return output

以上的`MLP`类中无须定义反向传播函数。系统将通过自动求梯度而自动生成反向传播所需的`backward`函数。

我们可以实例化`MLP`类得到模型变量`net`。下面的代码初始化`net`并传入输入数据`X`做一次前向计算。其中，`net(X)`将调用`MLP`类定义的`call`函数来完成前向计算。

In [5]:
x = tf.random.uniform((2,20))
net = MLP()
net(x)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.22419533, -0.16879301,  0.29826456,  0.3353717 ,  0.13821214,
         0.02322168,  0.03952552,  0.2131066 , -0.09013341, -0.12517235],
       [-0.41884312, -0.21783325,  0.24406308,  0.3132337 ,  0.09777944,
        -0.04760763,  0.10810249,  0.11710339, -0.03755157,  0.03344892]],
      dtype=float32)>

## 4.1.2 Sequential

我们刚刚提到，`tf.keras.Model`类是一个通用的部件。事实上，`Sequential`类继承自`tf.keras.Model`类。当模型的前向计算为简单串联各个层的计算时，可以通过更加简单的方式定义模型。这正是`Sequential`类的目的：它提供`add`函数来逐一添加串联的`Block`子类实例，而模型的前向计算就是将这些实例按添加的顺序逐一计算。

我们用Sequential类来实现前面描述的MLP类，并使用随机初始化的模型做一次前向计算。

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

model(x)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.14062548,  0.17708549, -0.34564868, -0.08045653, -0.21336536,
        -0.06365272,  0.0952149 , -0.14809619,  0.31732696,  0.1952827 ],
       [ 0.28201458,  0.27915776, -0.43677014, -0.21922818, -0.30286503,
        -0.21809286, -0.04954304, -0.26291206,  0.34253088,  0.13920398]],
      dtype=float32)>

## 4.1.3 build complex model

虽然`Sequential`类可以使模型构造更加简单，且不需要定义`call`函数，但直接继承`tf.keras.Model`类**可以极大地拓展模型构造的灵活性**。下面我们构造一个稍微复杂点的网络`FancyMLP`。在这个网络中，我们通过`constant`函数创建训练中不被迭代的参数，即常数参数。在前向计算中，除了使用创建的常数参数外，我们还使用`tensor`的函数和Python的控制流，并多次调用相同的层。

In [10]:
class FancyMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.rand_weight = tf.constant(
            tf.random.uniform((20,20)))
        self.dense = tf.keras.layers.Dense(units=20, activation=tf.nn.relu)
    
    def call(self, inputs):
        x = self.flatten(inputs)
        x = tf.nn.relu(tf.matmul(x, self.rand_weight) + 1)
        x = self.dense(x)
        while tf.norm(x) > 1:  # 计算向量、矩阵和张量的范数. 
            x /= 2
        if tf.norm(x) < 0.8:  
            x *= 10
        return tf.reduce_sum(x)

在这个`FancyMLP`模型中，我们使用了常数权重`rand_weight`（注意它不是模型参数）、做了矩阵乘法操作（`tf.matmul`）并重复使用了相同的`Dense`层。下面我们来测试该模型的随机初始化和前向计算。

In [11]:
net = FancyMLP()
net(x)

<tf.Tensor: shape=(), dtype=float32, numpy=18.029547>

因为`FancyMLP`和`Sequential`类都是`tf.keras.Model`类的子类，所以我们**可以嵌套调用它们**。

In [13]:
class NestMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.net = tf.keras.Sequential()
        self.net.add(tf.keras.layers.Flatten())
        self.net.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
        self.net.add(tf.keras.layers.Dense(32,activation=tf.nn.relu))
        self.dense = tf.keras.layers.Dense(units=16, activation=tf.nn.relu)
        
    def call(self,inputs):
        return self.dense(self.net(inputs))

net = tf.keras.Sequential()
net.add(NestMLP())
net.add(tf.keras.layers.Dense(20))
net.add(FancyMLP())

net(x)

<tf.Tensor: shape=(), dtype=float32, numpy=26.408482>

> 注：本节除了代码之外与原书基本相同，[原书传送门](https://zh.d2l.ai/chapter_deep-learning-computation/model-construction.html)

